<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parameter Sets
#### 기존 분석자료 기준으로 만든 최적 parameter - 다음 최적 parameter를 찾기 위한 initial input parameter로 이용

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
import pickle

def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

In [5]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [6]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [8]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [ ]:
params_sec = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
    "num_iterations" : [300], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.03, 0.04, 0.051, 0.06, 0.07], 
    "max_depth": [-1, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [1, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [0.3, 0.35, 0.4, 0.45, 0.5], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [None, 3, 4, 5],
    "max_delta_step": [0.4, 0.5, 0.6, 0.7, 0.8], # best value found, default = 0
    "scale_pos_weight": [2, 10, 15], # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
#     "is_unbalance": [True],  # 불균형 셋 조정. 주의: "scale_pos_weight"과 동시에 사용할 수 없음
    #---------------------------------------------
#     "scoring": ['accuracy', 'precision'], 
#     "early_stopping_rounds" : [200],
  
# ************* 아래 하나씩 테스트해서 취사 선택해야 함. **************
#     "bagging_frequency" : [5], 
#     "num_threads": [0], # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "min_data_in_leaf": [None], # 과적합 방지 파라미터
#     "lambda_l1": [0], # default 0
#     "lambda_l2": [0], #default 0
#     "min_gain_to_split": [None], # 분기하기 위해 필요한 최소한의 gain을 의미, 정규화시 사용
#     "num_threads": = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "max_bin": [32],  # data 준비할 때 사용하는 parameter     

}

In [ ]:
Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
[LightGBM] [Warning] Unknown parameter: eval_metric
Best Estimator: LGBMClassifier(

colsample_bytree=0.41, 
eval_metric='logloss',
learning_rate=0.004, 
max_delta_step=0.5, 
max_depth=2,
metric='loss', 
n_estimators=5, 
num_iterations=500, 
num_leaves=3,
objective='binary', 
random_state=42, 
subsample=0.005)
    
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.41, 'learning_rate': 0.004, 'max_depth': 2, 'metric': 'loss', 'n_estimators': 5, 'num_leaves': 3, 'objective': 'binary', 'subsample': 0.005}
Best Score: 0.9333333333333332, Best Index: 0
train accuracy: 0.8081, val accuracy 0.7442, test accuracy 0.7593
precision : 1.0000, recall : 0.2353, f1score : 0.3810, roc : 0.6176
[[37  0]
 [13  4]]

In [ ]:
Fitting 5 folds for each of 18000 candidates, totalling 90000 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.5 will be ignored. Current value: bagging_fraction=0.9
Best Estimator: LGBMClassifier(bagging_fraction=0.9, colsample_bytree=0.5, feature_fraction=0.8,
               learning_rate=0.05, max_delta_step=0.5, max_depth=2,
               metric='auc', num_iterations=500, num_leaves=4,
               objective='binary', random_state=42, subsample=0.5)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'metric': 'auc', 'n_estimators': 100, 'num_leaves': 4, 'objective': 'binary', 'subsample': 0.5}
Best Score: 0.8375180375180376, Best Index: 726
train accuracy: 1.0000, val accuracy 0.7907, test accuracy 0.8148
precision : 0.7059, recall : 0.7059, f1score : 0.7059, roc : 0.7854
[[32  5]
 [ 5 12]]

In [12]:
params_o = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
    "num_iterations" : [300], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.03, 0.04, 0.051, 0.06, 0.07], 
    "max_depth": [-1, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [1, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [0.3, 0.35, 0.4, 0.45, 0.5], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [None, 3, 4, 5],
    "max_delta_step": [0.4, 0.5, 0.6, 0.7, 0.8], # best value found, default = 0
    "scale_pos_weight": [2, 10, 15], # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
#     "is_unbalance": [True],  # 불균형 셋 조정. 주의: "scale_pos_weight"과 동시에 사용할 수 없음
    #---------------------------------------------
#     "scoring": ['accuracy', 'precision'], 
#     "early_stopping_rounds" : [200],
  
# ************* 아래 하나씩 테스트해서 취사 선택해야 함. **************
#     "bagging_frequency" : [5], 
#     "num_threads": [0], # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "min_data_in_leaf": [None], # 과적합 방지 파라미터
#     "lambda_l1": [0], # default 0
#     "lambda_l2": [0], #default 0
#     "min_gain_to_split": [None], # 분기하기 위해 필요한 최소한의 gain을 의미, 정규화시 사용
#     "num_threads": = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
#     "max_bin": [32],  # data 준비할 때 사용하는 parameter     

}

In [13]:
params_o = {
    "boosting_type" : ['gbdt'], # ['dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
#     "num_iterations" : [200], #[450, 500, 550], #"num_iterations" 
    "learning_rate": [0.06, 0.065, 0.07, 0.075, 0.08], 
    "max_depth": [-1, 2, 3, 4], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
    "num_leaves": [2, 3, 4, 5, 6], #  # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
    "subsample": [1, 0.001, 0.002],# 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
    "colsample_bytree": [0.3, 0.35, 0.4, 0.45, 0.5], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
    "objective": ['binary'],
    "metric": ['binary_logloss'], # "metric": ['loss'], # mae : mean absolute error, mse : mean squared error, 
                      # binary_logloss : loss for binary classification, multi_logloss : loss for multi classification

    "n_estimators": [80, 100, 120],
    "max_delta_step": [ 0.8, 0.9], # best value found, default = 0
    "scale_pos_weight": [10], # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
#     "scoring": ['accuracy', 'precision'],   

}

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [ ]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.

In [ ]:
# # 최초의 empty df 생성
# df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [ ]:
df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), 1)
result_dict = calc_results(lgbmgs.best_estimator_, train_scaled, val_scaled, test_scaled, test_target)
df_result = make_df_from_estimator(result_dict, 1)
df_concat = pd.concat([df_estimator, df_result])

In [ ]:
df_concat

In [ ]:
df_concat.index =['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type',
       'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight',
       'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective',
       'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample',
       'subsample_for_bin', 'subsample_freq', 'max_delta_step', 'metric',
       'num_iterations', 'scale_pos_weight', 'best_score', 'best_index',
       'acc_train', 'acc_val', 'acc_test', 'precision', 'recall', 'f1score',
       'roc', 'tn', 'fp', 'fn', 'tp']
df_concat

In [ ]:

num_leaves = 20~3000
max_depth = 3 ~12
learning_rate = 0.01 ~ 0.3

parameters = {'learning_rate': [0.01,0.02,0.03],
                  'subsample'    : [0.9, 0.5, 0.2],
                  'n_estimators' : [100,500,1000],
                  'max_depth'    : [4,6,8]
                 }

bayesian optimization 확인